In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import emoji
from sklearn.feature_extraction.text import CountVectorizer #TF
from sklearn.feature_extraction.text import TfidfVectorizer #Tf-idf

In [2]:
df = pd.read_excel("DATOS_UNICOS_TOTALES.xlsx")
df.head()

,HATE,COMENTARIO
0,0.0,¿Ya vieron que en la #AO habrá evento por el #...
1,0.0,muy XD que para el primer punto exiges que Yam...
2,0.0,Apoco lees?\n1-El primer punto es para hablar ...
3,0.0,vos se nota que no tenes compresión lectora xD...
4,0.0,"No, literalmente Kaido y todos los piratas bes..."


In [3]:
def clean(text):
    sw = stopwords.words('spanish')
    text = text.lower()
    text = emoji.demojize(text)
    urls = []
    for i in text.split():
        if not i.startswith('http'):
            urls.append(i)
    text1 = ' '.join(urls)
    cl1 = re.findall(r'[^@#&\w+][\w+]+',text1)
    text2 = ' '.join(cl1)
    cl3 = re.findall(r'[a-zA-Záéíóúñ\s:_]+',text2)
    text3 = ' '.join(cl3)
    c = text3.maketrans('áéíóú','aeiou')
    text4 = text3.translate(c)   
    L = [palabra for palabra in text4.split() if len(palabra) >2 and palabra not in sw]
    return ' '.join(L)

## Limpiando Data

In [4]:
list_limpios = []
for comment in df['COMENTARIO']:
    list_limpios.append(clean(str(comment)))

In [5]:
df['LIMPIOS'] = list_limpios
df.head()

,HATE,COMENTARIO,LIMPIOS
0,0.0,¿Ya vieron que en la #AO habrá evento por el #...,vieron habra evento enserio rifan mostrando em...
1,0.0,muy XD que para el primer punto exiges que Yam...,primer punto exiges yamato autoproclame hombre...
2,0.0,Apoco lees?\n1-El primer punto es para hablar ...,lees primer punto hablar yamato usa pronombres...
3,0.0,vos se nota que no tenes compresión lectora xD...,nota tenes compresion lectora diciendo hipocri...
4,0.0,"No, literalmente Kaido y todos los piratas bes...",literalmente kaido piratas bestia incluso luff...


### Define

In [162]:
df_ofensivo = df[df["HATE"] == 1]
df_noOfensivo = df[df["HATE"] == 0]

In [163]:
df_ofensivo

,HATE,COMENTARIO,LIMPIOS
15,1.0,Esto es la máxima imbecilidad,maxima imbecilidad
46,1.0,No nos engañemos. Los homosexuales bien sabemo...,engañemos homosexuales bien sabemos union pers...
52,1.0,"Claro que es ilegal.\nSi esa fuera la razón, c...",ilegal razon claro ademas inmoral socialmente ...
89,1.0,A los curas pro MOVIMIENTO LGTBI+ deberían ser...,curas pro movimiento lgtbi deberian ser server...
90,1.0,"A los de siempre. Hablo del Movimiento, de la ...",siempre hablo movimiento apologia lgtbi person...
...,...,...,...
14157,1.0,@13Azulcrema Y eso que cabrón? Si allá está pe...,cabron alla penado ser gay tambien vas ser hom...
14181,1.0,"Cuchame, soy la mierda más delulu que te puede...",mierda mas delulu puedes encontrar adicta gay ...
14182,1.0,@NEXWAL999 tampoco te las harías pq eres gay,tampoco harias gay
14203,1.0,Not mi amiga diciéndome que ha ido a comer sol...,amiga diciendome ido comer sola maricones meti...


### Train and Teste (80 - 20)

In [164]:
ofen_train = df_ofensivo[:403].sample(400)
ofen_train.dropna(how = 'any',inplace=True)
ofen_train.shape

(400, 3)

In [165]:
ofen_teste = df_ofensivo[400:].sample(100)
ofen_teste.dropna(how = 'any', inplace = True)
ofen_teste.shape

(100, 3)

In [166]:
NoOfen_train = df_noOfensivo[:3000].sample(400)
NoOfen_train.dropna(how = 'any',inplace=True)
NoOfen_train.shape

(400, 3)

In [167]:
NoOfen_teste = df_noOfensivo[3000:5000].sample(100)
NoOfen_teste.dropna(how = 'any', inplace = True)
NoOfen_teste.shape

(100, 3)

## TF - IDF

In [171]:
ofensivos = ofen_train['LIMPIOS'].values.tolist()
#TF-IDF
tf_idf = TfidfVectorizer(max_features = 50)
ofen_train_tfidf = tf_idf.fit_transform(ofensivos)
ofen_train_tfidf.shape

(400, 50)

In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [173]:
print(tf_idf.get_feature_names())

['ahora', 'asco', 'asi', 'bandera', 'callate', 'cara', 'comunidad', 'dice', 'dios', 'enfermedad', 'enfermos', 'face_with_tears_of_joy', 'familia', 'gay', 'gays', 'gente', 'hombre', 'homosexual', 'lgbt', 'madre', 'mal', 'marica', 'maricon', 'mas', 'matrimonio', 'mexico', 'mierda', 'mismo', 'movimiento', 'nauseated_face', 'organizacion', 'palabra', 'peor', 'personas', 'puede', 'pueden', 'puta', 'puto', 'putos', 'quieren', 'rainbow_flag_selector', 'ser', 'solo', 'sos', 'tan', 'tener', 'turkey', 'vas', 'ver', 'vida']


In [183]:
noOfensivos = NoOfen_train['LIMPIOS'].values.tolist()
#TF-IDF
tf_idf = TfidfVectorizer(max_features = 50)
NoOfen_train_tfidf = tf_idf.fit_transform(noOfensivos)
NoOfen_train_tfidf.shape

(400, 50)

In [184]:
ofensivos_test = ofen_teste['LIMPIOS'].values.tolist()
#TF-IDF
tf_idf = TfidfVectorizer(max_features = 50)
ofen_teste_tfidf = tf_idf.fit_transform(ofensivos_test)
ofen_teste_tfidf.shape

(100, 50)

In [185]:
NoOfensivos_test = NoOfen_teste['LIMPIOS'].values.tolist()
#TF-IDF
tf_idf = TfidfVectorizer(max_features = 50)
NoOfen_teste_tfidf = tf_idf.fit_transform(NoOfensivos_test)
NoOfen_teste_tfidf.shape

(100, 50)

## TF

In [186]:
ofensivos = ofen_train['LIMPIOS'].values.tolist()
#TF
tf = CountVectorizer(max_features = 50)
ofen_train_tf = tf.fit_transform(ofensivos)
ofen_train_tf.shape

(400, 50)

In [187]:
print(tf.get_feature_names())

['ahora', 'asco', 'asi', 'bandera', 'callate', 'cara', 'comunidad', 'dice', 'dios', 'enfermedad', 'enfermos', 'face_with_tears_of_joy', 'familia', 'gay', 'gays', 'gente', 'hombre', 'homosexual', 'lgbt', 'madre', 'mal', 'marica', 'maricon', 'mas', 'matrimonio', 'mexico', 'mierda', 'mismo', 'movimiento', 'nauseated_face', 'organizacion', 'palabra', 'peor', 'personas', 'puede', 'pueden', 'puta', 'puto', 'putos', 'quieren', 'rainbow_flag_selector', 'ser', 'solo', 'sos', 'tan', 'tener', 'turkey', 'vas', 'ver', 'vida']


In [188]:
noOfensivos = NoOfen_train['LIMPIOS'].values.tolist()
#TF
tf = CountVectorizer(max_features = 50)
NoOfen_train_tf = tf.fit_transform(noOfensivos)
NoOfen_train_tf.shape

(400, 50)

In [189]:
ofensivos_test = ofen_teste['LIMPIOS'].values.tolist()
#TF
tf = CountVectorizer(max_features = 50)
ofen_teste_tf = tf.fit_transform(ofensivos_test)
ofen_teste_tf.shape

(100, 50)

In [190]:
NoOfensivos_test = NoOfen_teste['LIMPIOS'].values.tolist()
#TF
tf = CountVectorizer(max_features = 50)
NoOfen_teste_tf = tf.fit_transform(NoOfensivos_test)
NoOfen_teste_tf.shape

(100, 50)

### Concatenando data tf-idf

In [191]:
X_train = np.concatenate((ofen_train_tfidf.toarray().tolist(), NoOfen_train_tfidf.toarray().tolist() ), axis=0)
X_train.shape

(800, 50)

In [192]:
Y_train = np.concatenate((ofen_train['HATE'].values, NoOfen_train['HATE'].values), axis = 0)
Y_train.shape

(800,)

In [193]:
X_teste = np.concatenate((ofen_teste_tfidf.toarray().tolist(),NoOfen_teste_tfidf.toarray().tolist()), axis=0)
X_teste.shape

(200, 50)

In [194]:
Y_teste = np.concatenate((ofen_teste['HATE'].values, NoOfen_teste['HATE']), axis = 0)
Y_teste.shape

(200,)

### Concatenando data TF

In [195]:
X_train_tf = np.concatenate((ofen_train_tf.toarray().tolist(), NoOfen_train_tf.toarray().tolist() ), axis=0)
X_train_tf.shape

(800, 50)

In [196]:
Y_train_tf = np.concatenate((ofen_train['HATE'].values, NoOfen_train['HATE'].values), axis = 0)
Y_train_tf.shape

(800,)

In [197]:
X_teste_tf = np.concatenate((ofen_teste_tf.toarray().tolist(),NoOfen_teste_tf.toarray().tolist()), axis=0)
X_teste_tf.shape

(200, 50)

In [198]:
Y_teste_tf = np.concatenate((ofen_teste['HATE'].values, NoOfen_teste['HATE'].values), axis = 0)
Y_teste_tf.shape

(200,)

# Model

In [199]:
from sklearn import svm
from sklearn.metrics import accuracy_score,classification_report


### Tf-IDF

In [215]:
clasificador = svm.SVC(gamma=0.001,C=10000)
clasificador.fit(X_train,Y_train)

SVC(C=10000, gamma=0.001)

In [216]:
Y_pred = clasificador.predict(X_teste)

In [217]:
accuracy_score(Y_teste,Y_pred)

0.69

In [218]:
print(classification_report(Y_teste,Y_pred))

              precision    recall  f1-score   support

         0.0       0.64      0.88      0.74       100
         1.0       0.81      0.50      0.62       100

    accuracy                           0.69       200
   macro avg       0.72      0.69      0.68       200
weighted avg       0.72      0.69      0.68       200



### TF

In [228]:
clasificador = svm.SVC(gamma=0.001,C=100000)
clasificador.fit(X_train_tf,Y_train_tf)

SVC(C=100000, gamma=0.001)

In [229]:
Y_pred_tf = clasificador.predict(X_teste_tf)

In [230]:
accuracy_score(Y_teste_tf,Y_pred_tf)

0.795

In [231]:
print(classification_report(Y_teste_tf,Y_pred_tf))

              precision    recall  f1-score   support

         0.0       0.75      0.89      0.81       100
         1.0       0.86      0.70      0.77       100

    accuracy                           0.80       200
   macro avg       0.81      0.79      0.79       200
weighted avg       0.81      0.80      0.79       200



## Matriz de Confusion

In [232]:
from sklearn.metrics import confusion_matrix

In [233]:
confusion_matrix(Y_teste, Y_pred)

array([[88, 12],
       [50, 50]], dtype=int64)

In [234]:
confusion_matrix(Y_teste_tf, Y_pred_tf)

array([[89, 11],
       [30, 70]], dtype=int64)